<a href="https://colab.research.google.com/github/RiemanBall/Machine-Learning/blob/master/NaiveBayesClassifier/NaiveBayesClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification with Naive Bayes
This notebook will build 
1. a Gaussian Naive Bayes Classifier from scratch with Iris dataset, and
2. a Multinomial Naive Bayes Classifier from scratch to classify spam email.

In [1]:
from math import ceil
from collections import defaultdict, Counter
from typing import List, DefaultDict, Tuple

import numpy as np
np.random.seed(0)
from scipy.stats import norm
from scipy.special import logsumexp

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import GaussianNB

import matplotlib.pyplot as plt
%matplotlib inline

## Naive Bayes For Binary Classification

In binary classification, given the data feature $x \in R^m $, we want to classify its class. That is,

\begin{equation}
P(C_1|x)>0.5?
\end{equation}

By Bayes Theorem,

\begin{equation}
P(C_1|x) = \frac{P(x|C_1)P(C_1)}{P(x)}\propto P(x|C_1)P(C_1)
\end{equation}

\\
where $P(x)$ is a normalization term, $P(C_1)$ can be treated as prior or approximated by sample distribution. The difficult question is how to get $P(x|C_1)$, and it is where the naive assumptions come in to form a Naive Bayes Classifier.

Originally, we have assumptions:
1. Label classes are independent,
2. Samples are i.i.d.,

and now we add one more naive assumption:
3. Feature values are independent given the label (conditionally independent)

In this case, we can rewrite $P(x|C_1)$ as

\begin{equation}
P(x|C_1)=\prod^m_k{P(x_k|C_1)}
\end{equation}

where $x_k$ is the $k$-th feature value in the sample.

## Gaussian Naive Bayes

### Preprocessing data
- Read data
- Split to training set and validation set

In [2]:
iris = load_iris()
X = iris.data
Y = iris.target

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 23, test_size = 0.4)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(90, 4)
(60, 4)
(90,)
(60,)


### Build A Gaussian Naive Bayes Classifier

A generative model usually predefines the underlying distribution. Here we assume the probabilistic distribution of $P(x_k|C_1)$ is Gaussian with mean $\mu_k$ and standard deviation $\sigma_k$.

Given a dataset $X\in R^{n\times m}$, we want to find the optimal $\mu_k$ and $\sigma_k$ to explain the sample distribution the best. That is, maximum likelihood estiamte (MLE)

\begin{equation}
\text{maximize}\ \text{likelihood} = P(C_1|X \in \{x | C_x = C_1\})
\end{equation}
\
by taking log and combining with naive bayes assumption, this is equivalent to

\begin{equation}
\text{maximize}\ \sum^m_k{\log{P(X_k|C_1)}} \\
= \sum^m_k{\text{maximize}_{\mu_k, \sigma_k}\ log{N(X_k|\mu_k, \sigma_k)}}
\end{equation}
\
where $N(X_k|\mu_k, \sigma_k)$ is the gaussian distribution of $k$-th feature in the dataset. With the assumption of i.i.d. samples,

\begin{equation}
\log{N(X_k|\mu_k, \sigma_k)} = \sum^n_i{\log{N(x^i_k|\mu_k, \sigma_k)}}
\end{equation}

\\
We will skip the derivation of computing the optimal $\mu_k$ and $\sigma_k$ here, but the result is using the sample mean and sample standard deviation, respectively.

To avoid underflow, we compute the log probability 

\begin{equation}
\log{(P(x|C_1)P(C_1))}
= \sum^m_k{\{\log{N(x_k|\mu_k, \sigma_k)}\}} + \log{P(C_1)}\\
\log{P(x)} 
= \log{\sum^C_c{(P(x|C_c)P(C_c))}} 
\end{equation}



In [4]:
class GNB:
    def __init__(self):
        self.means_by_class = None
        self.stds_by_class  = None
        self.prior_by_class = None
        self.unique_classes = None


    def get_prior_by_class(self, Y):
        '''
        Compute the prior of each class based on the sample distribution
        Inputs:
            Y: (N, ) ndarray. N data labels

        Outputs:
            prior_by_class: defaultdict(key=int, val=float). The key is the label id, and the value 
                            is the prior belonging to the label class.
        '''
        self.unique_classes = sorted(set(Y))
        self.prior_by_class = defaultdict(float)
        cnt = Counter(Y)
        total = sum(cnt.values())
        for label in self.unique_classes:
            self.prior_by_class[label] = cnt[label] / total


    def separate_data_by_class(self, X: np.ndarray, Y: np.ndarray)->DefaultDict:
        '''
        Separate samples by class and store them to dictionary type variable
        Inputs:
            X: (N,m) ndarray. N data, m features
            Y: (N, ) ndarray. N labels
        Outputs:
            X_by_class: defaultdict(key=int, val=(N_c, m) ndarray). The key is the label number,
                        and the value is the samples belonging to the label class. N_c is the number
                        of samples in class c
        '''
        X_by_class = defaultdict(list)

        for x, y in zip(X, Y):
            X_by_class[y].append(x)

        for label, data_subset in X_by_class.items():
            X_by_class[label] = np.array(data_subset)

        return X_by_class


    def compute_mean_std_by_class(self, X_by_class: DefaultDict):
        '''
        Compute the mean and standard deviation of each feature in the data by class
        Inputs:
            X_by_class: defaultdict(key=int, val=(N_c, m) ndarray). The key is the label number,
                        and the value is the samples belonging to the label class. N_c is the number
                        of samples in class c
        Outputs:
            self.means_by_class: defaultdict(key=int, val=(m, ) ndarray). The key is the label number,
                                 and the value is the (m, ) array of m means of each feature belonging 
                                 to the label class.
            self.stds_by_class:  defaultdict(key=int, val=(m, ) ndarray). The key is the label number,
                                 and the value is the (m, ) array of m standard deviations of each feature 
                                 belonging to the label class.
        '''
        self.means_by_class = defaultdict(float)
        self.stds_by_class  = defaultdict(float)
        for label, data_subset in X_by_class.items():
            self.means_by_class[label] = np.mean(data_subset, axis=0)
            self.stds_by_class[label]  = np.std(data_subset, axis=0)
            if self.means_by_class[label].shape[0] != data_subset.shape[1]:
                print("Wrong dimension!")


    def fit(self, X: np.ndarray, Y: np.ndarray):
        '''
        Fit the Naive Bayes Model with input feature X and label Y
        Inputs:
            X: (N,m) ndarray. N data, m features
            Y: (N, ) ndarray. N labels
        '''
        self.get_prior_by_class(Y)

        # Separate X by class
        X_by_class = self.separate_data_by_class(X, Y)
        # Compute the optimal mean and std for each
        self.compute_mean_std_by_class(X_by_class)


    def log1DGaussian(self, X, mean=0.0, std=1.0):
        '''
        Compute the log of normal distribution of x. Support single feature or one sample point
        Inputs:
            X: (N, m) ndarray. N = number of data, m = number of features
            mean: (m, ) ndarray. m = number of features
            std: (m, ) ndarray. m = number of features

        Outputs:
            log of normal distribution: (N, m) ndarray
        '''
        return -np.square((X - mean) / std) / 2 - np.log(np.sqrt(2 * np.pi) * std)


    def compute_all_jll(self, X):
        '''
        Compute joint log likelihood of X for all classes
        Input:
            X: (N, m) ndarray. N = number of data, m = number of features

        Output:
            jll_all: (N, C) ndarray. N = number of data, C = number of classes
        '''
        jll_all = []

        for label in self.unique_classes:
            log_prior = np.log(self.prior_by_class[label])
            means = self.means_by_class[label]  # shape=(m, )
            stds  = self.stds_by_class[label]   # shape=(m, )
            log_cond_prob = np.sum(self.log1DGaussian(X, means, stds), axis = 1) # shape=(N, )
            jll_all.append(log_cond_prob + log_prior)
            
        jll_all = np.array(jll_all).T

        return jll_all


    def pred_log_prob(self, X):
        '''
        Compute log probability of X for all classes
        Input:
            X: (N, m) ndarray. N = number of data, m = number of features

        Output:
            log_prob: (N, C) ndarray. N = number of data, C = number of classes
        '''

        num_data, num_features = X.shape
        num_classes = len(self.unique_classes)

        jll_all = self.compute_all_jll(X)  # shape = (num_data, num_classes)

        log_p_x = logsumexp(jll_all, axis = 1).reshape(-1, 1) # shape = (num_data, 1)

        return jll_all - log_p_x


    def pred_prob(self, X: np.ndarray)->np.ndarray:
        '''
        Compute probability of X for all classes
        Input:
            X: (N, m) ndarray. N = number of data, m = number of features

        Output:
            prob: (N, C) ndarray. N = number of data, C = number of classes
        '''
        return np.exp(self.pred_log_prob(X))


    def predict(self, X: np.ndarray)->Tuple[np.ndarray, np.ndarray]:
        '''
        Predict each class of input X.
        Inputs:
            X: (N, m) ndarray.
        Outputs:
            pred_classes: (N, ) ndarray. N = number of data
            pred_probs: (N, C) ndarray. N = number of data, C = number of classes
        '''
        pred_probs   = self.pred_prob(X)
        pred_classes = np.argmax(pred_probs, axis = 1)

        return pred_classes, pred_probs

### Train GNB

In [5]:
gnb = GNB()
gnb.fit(X_train, Y_train)

In [6]:
for label in gnb.unique_classes:
    print(f"Label: {label}")
    print(f"mean: {gnb.means_by_class[label]}")
    print(f"std: {gnb.stds_by_class[label]}")
    print(f"prior: {gnb.prior_by_class[label]}\n")

Label: 0
mean: [4.95862069 3.37931034 1.4137931  0.24827586]
std: [0.37279674 0.39599661 0.15914456 0.11925253]
prior: 0.32222222222222224

Label: 1
mean: [5.921875 2.746875 4.26875  1.3375  ]
std: [0.48652748 0.32690248 0.48633675 0.20425168]
prior: 0.35555555555555557

Label: 2
mean: [6.5137931  2.93103448 5.50689655 2.03448276]
std: [0.5322199  0.27556018 0.51053119 0.26428284]
prior: 0.32222222222222224



#### Prediction on Training and Testing data

In [7]:
pred_classes_train, pred_probs_train = gnb.predict(X_train)

#### Compare with Scikit Learn

In [8]:
gnb_skl = GaussianNB()
gnb_skl.fit(X_train, Y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [9]:
for label, (mean, var, prior) in enumerate(zip(gnb_skl.theta_, gnb_skl.sigma_, gnb_skl.class_prior_)):
    print(f"Label: {label}")
    print(f"mean: {mean}")
    print(f"std: {np.sqrt(var)}")
    print(f"prior: {prior}\n")

Label: 0
mean: [4.95862069 3.37931034 1.4137931  0.24827586]
std: [0.37279674 0.39599662 0.15914457 0.11925254]
prior: 0.32222222222222224

Label: 1
mean: [5.921875 2.746875 4.26875  1.3375  ]
std: [0.48652748 0.32690249 0.48633676 0.20425169]
prior: 0.35555555555555557

Label: 2
mean: [6.5137931  2.93103448 5.50689655 2.03448276]
std: [0.53221991 0.27556019 0.51053119 0.26428285]
prior: 0.32222222222222224



In [10]:
pred_classes_train_skl = gnb_skl.predict(np.atleast_2d(X_train))
pred_probs_train_skl = gnb_skl.predict_proba(np.atleast_2d(X_train))

In [11]:
print(f"Differece of predicted labels: {pred_classes_train_skl - pred_classes_train}")
print(f"Differece of predicted probabilities: \n{np.sum(pred_probs_train_skl - pred_probs_train, axis = 1)}")


Differece of predicted labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Differece of predicted probabilities: 
[ 4.16333634e-17  7.23872378e-22 -1.71727460e-16  3.38846320e-17
  7.49285345e-17  2.45202486e-16  2.63677968e-16 -1.08994929e-16
 -3.73456826e-17  1.66533454e-16  8.94037814e-25  1.38172392e-19
 -3.11199759e-16  2.11433905e-22  3.34673901e-22  2.15641036e-16
 -1.38777878e-16 -9.88385805e-17  6.94892397e-23  1.62413485e-16
  1.53089347e-16 -1.11455983e-16  3.27650343e-17 -3.06401475e-16
 -2.19442520e-16  6.00617854e-24  3.14418630e-18  3.46944695e-18
 -3.88578059e-16 -1.01728233e-16  1.88770151e-22  7.80625564e-17
  6.93889390e-17 -1.79896245e-16 -1.38777878e-16  1.66533454e-16
 -1.62774745e-16 -1.38777878e-16 -2.43300050e-16  5.42800073e-17
  8.94405550e-24 -1.69077517e-16  2.55575516e-17  3.34309006e-25
  3.15293152e-16 -5.55111512e-1

### Predict on the test set

In [12]:
pred_classes_test, pred_probs_test = gnb.predict(X_test)

In [13]:
label_names = ['Setosa', 'Versicolor', 'Virginica']
print(classification_report(Y_test, pred_classes_test, target_names = label_names))

              precision    recall  f1-score   support

      Setosa       1.00      1.00      1.00        21
  Versicolor       0.85      0.94      0.89        18
   Virginica       0.95      0.86      0.90        21

    accuracy                           0.93        60
   macro avg       0.93      0.93      0.93        60
weighted avg       0.94      0.93      0.93        60

